In [1]:
# Clone the repository
!git clone https://github.com/huggingface/lerobot.git
%cd lerobot

# Install the library in editable mode with training dependencies
!pip install "transformers==4.57.6"
!pip install -e .
!pip install wandb  # Highly recommended for monitoring your FLL progress


In [6]:
# Setting up env
import os
from google.colab import userdata

# WANDB env
import wandb
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
wandb.login()
print("login to wandb")

# HUGGINGFACE env
from huggingface_hub import login
login(token=userdata.get('HF_TOKEN'))
print("login to hf")


In [7]:
# Adjust the following variables before starting training
tag = "350episodes"
hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_training_combined"
model_repo = f"{hf_namespace}/fll_diffusion_model_colab"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 1000
batch_size = 16
num_workers = 2

# Clean up output from last run
!rm -rf {local_output}

!python src/lerobot/scripts/lerobot_train.py \
    --policy.type=diffusion \
    --dataset.repo_id={training_repo} \
    --dataset.revision={tag} \
    --job_name=flamebots_training \
    --output_dir={local_output} \
    --policy.device=cuda \
    --steps={steps} \
    --save_freq=1000 \
    --batch_size={batch_size} \
    --log_freq=1000 \
    --num_workers={num_workers} \
    --policy.use_amp=true \
    --policy.push_to_hub=true \
    --policy.repo_id={model_repo} \
    --wandb.enable=true

#!hf repo tag create {model_repo} {tag} -m "Diffusion model trained on {training_repo}, tag {tag}"